In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from utils_training import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [2]:
N_train = 1000
N_bound = 200

# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)

x_inside = traindata[:,0:1]
t_inside = traindata[:,1:2]

x_inside = numpy_to_tensor(x_inside, var_name="x_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
t_inside = numpy_to_tensor(t_inside, var_name="t_inside", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

x_bound = lb[0]+(la[0]-lb[0])*lhs(1,N_bound)
t_bound = lb[1]+(la[1]-lb[1])*lhs(1,N_bound)

x_bound = numpy_to_tensor(x_bound, var_name="x_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)
t_bound = numpy_to_tensor(t_bound, var_name="t_bound", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = False)

x_inside: Column 0: range from -0.9996169610992423 to 0.9999084330058925
t_inside: Column 0: range from 0.0006221087710398319 to 0.9998795647581709
x_bound: Column 0: range from -0.9974393487799895 to 0.9951859062149828
t_bound: Column 0: range from 0.0010549176968800646 to 0.9999568866549661


In [3]:
def exact_func(x_input):
    x_value = x_input[:, 0:1]
    t_value = x_input[:, 1:2]
    
    return  np.exp(-t_value) * np.sin(np.pi * x_value)

In [4]:
random_seed = 1234
np.random.seed(random_seed)
observe_number = 10

observe_data = lb+(la-lb)*lhs(2,observe_number)
observe_clear_u = exact_func(observe_data)

observe_u = observe_clear_u

observe_data = numpy_to_tensor(observe_data, var_name="observe_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_clear_u = numpy_to_tensor(observe_clear_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
observe_u = numpy_to_tensor(observe_u, var_name="observe_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
print('J:',len(observe_u))

observe_data_x_inside = observe_data[:,0:1]
observe_data_t_inside = observe_data[:,1:2]

observe_data: Column 0: range from -0.9616961099242215 to 0.9545653243224748
observe_data: Column 1: range from 0.06221087710398319 to 0.9882641190636117
observe_u: Column 0: range from -0.4622391171270676 to 0.6941402599252228
observe_u: Column 0: range from -0.4622391171270676 to 0.6941402599252228
J: 10


In [5]:
np.random.seed(5678)
N_test_number = 10000

test_data = lb+(la-lb)*lhs(2,N_test_number)
test_u = exact_func(test_data)

test_data = numpy_to_tensor(test_data, var_name="test_data", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)
test_u = numpy_to_tensor(test_u, var_name="test_u", value_range_dim = True, to_torch = True, to_cuda = True, requires_grad = True)

test_data_x_inside = test_data[:,0:1]
test_data_t_inside = test_data[:,1:2]

test_data: Column 0: range from -0.9999021346039978 to 0.999818369091213
test_data: Column 1: range from 5.933244265166393e-06 to 0.9999791490405054
test_u: Column 0: range from -0.9962151573581175 to 0.9967165533751065


In [6]:
def output_transform(x_input, y_input):
    x_in = x_input[:, 0:1]
    t_in = x_input[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * y_input + torch.sin(np.pi * x_in)

In [7]:
C1 = torch.tensor(2.0, requires_grad=True)

In [8]:
def get_loss_f(x_grad,t_grad,PINNs,C,return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_grad,t_grad),axis=1))
    E_inside = output_transform(torch.cat((x_grad,t_grad),axis=1),E_inside)
    
    u_xx = compute_higher_order_derivatives(E_inside, [x_grad,x_grad])
    u_t = compute_higher_order_derivatives(E_inside, [t_grad])
    
    loss_f_sequence = u_t-C*u_xx+torch.exp(-t_grad)*(torch.sin(torch.tensor(np.pi)*x_grad)-torch.tensor(np.pi)*torch.tensor(np.pi)*torch.sin(torch.tensor(np.pi)*x_grad))
    loss_f_squared_sequence = torch.square(loss_f_sequence)

    if return_sequence=='yes':
        return loss_f_squared_sequence
    else:
        return torch.mean(loss_f_squared_sequence)

In [9]:
def get_loss_bound(bound_x, bound_t, PINNs, C, return_sequence='not'):
    
    E_bound_x_zero = PINNs(torch.cat((bound_x,torch.zeros_like(bound_x)),axis=1)) 
    Exact_x_zero = torch.sin(torch.tensor(np.pi)*bound_x)
    loss_bound_for_a = torch.mean(torch.square(E_bound_x_zero-Exact_x_zero))
    
    E_bound_fu_1_t = PINNs(torch.cat((-torch.ones_like(bound_t),bound_t),axis=1)) 
    loss_bound_for_b = torch.mean(torch.square(E_bound_fu_1_t))
    
    E_bound_1_t = PINNs(torch.cat((torch.ones_like(bound_t),bound_t),axis=1))
    loss_bound_for_c = torch.mean(torch.square(E_bound_1_t))
    
    loss_bound_value = loss_bound_for_a+loss_bound_for_b+loss_bound_for_c
    
    return loss_bound_value

In [10]:
#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [11]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

net_settings_for_PINNs1 = NetSetting(input_dims=2, hidden_neurons_list=[20]*4, 
                                     output_dims=1, hidden_activation='tanh', 
                                     output_activation=None, initializer_method='xavier')
PINNs1 = get_mlp_pinn(net_settings_for_PINNs1)
PINNs1.cuda()

initialize_weights(PINNs1, net_settings_for_PINNs1.initializer_method)

optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)    
optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

In [12]:
x_inside_all = torch.cat((x_inside,observe_data[:,0:1]),axis=0)
t_inside_all = torch.cat((t_inside,observe_data[:,1:2]),axis=0)

In [13]:
net_settings_for_discriminator1 = NetSetting(input_dims=3, hidden_neurons_list=[20]*2, 
                                     output_dims=1, hidden_activation='sigmoid(tanh)', 
                                     output_activation='sigmoid(tanh)', initializer_method='xavier')

discriminator1 = get_mlp_pinn(net_settings_for_discriminator1)
discriminator1.cuda() 
initialize_weights(discriminator1, net_settings_for_discriminator1.initializer_method)

optimizer2 = optim.Adam(discriminator1.parameters(), lr=1e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)

In [14]:
def train_optimizer1(PINNs,C,discriminator):
    
    for param in PINNs.parameters():
        param.requires_grad = True
    
    C.requires_grad = True
    
    for param in discriminator.parameters():
        param.requires_grad = False

In [15]:
def train_optimizer2(PINNs,C,discriminator):
    
    for param in PINNs.parameters():
        param.requires_grad = False
    
    C.requires_grad = False
    
    for param in discriminator.parameters():
        param.requires_grad = True

In [16]:
def train_for_gan(PINNs,C,discriminator,observe_data_input,observe_u_input):
    
    
    ############ loss D ###########
    train_optimizer2(PINNs,C,discriminator)
    
    pre_H = PINNs(observe_data_input)
    pre_H = output_transform(observe_data_input,pre_H)
    
    d_fake = discriminator(torch.cat((observe_data_input,pre_H),1))
    d_real = discriminator(torch.cat((observe_data_input,observe_u_input),1))

    loss_D = torch.mean(1-d_real)+torch.mean(d_fake)

    optimizer2.zero_grad()
    loss_D .backward()
    optimizer2.step()  

    ############ loss G ###########
    train_optimizer1(PINNs,C,discriminator)
    
    pre_H = PINNs(observe_data_input)
    pre_H = output_transform(observe_data_input,pre_H)
    
    d_fake = discriminator(torch.cat((observe_data_input,pre_H),1))

    loss_G = torch.mean(1-d_fake)+torch.mean(torch.square(pre_H - observe_u_input))

    optimizer1.zero_grad()
    loss_G.backward()
    optimizer1.step()  
    
    return PINNs,C,discriminator

In [17]:
############## Record list ###############
loss_all_1 = []
loss_f_1 = []
loss_f_for_collocation_1 = []
loss_f_for_T_1 = []
loss_f_excapt_T_1 = []
loss_T_1 = []
loss_T_clear_1 = []
loss_T_1_test_data = []
test_loss_1 = []
C1_list = []
############## Record list ###############

nIter1 = 10000
it = 0

while it<nIter1:
    
    if it<6000:
        ######### GAN  #########
        PINNs1,C1,discriminator1 = train_for_gan(PINNs1,C1,discriminator1,observe_data,observe_u)  
        
    #########loss f#########    
    loss_f = get_loss_f(x_inside_all,t_inside_all,PINNs1,C1,return_sequence='not')

    #########loss observation#########        
    E_observation = PINNs1(observe_data) 
    E_observation = output_transform(observe_data,E_observation)
    loss_observation = torch.mean(torch.square(E_observation-observe_u))    

    #########loss PI#########
    loss = loss_f+10*loss_observation
    
    #########test_loss NRMSE#########
    pre_u = PINNs1(test_data)
    pre_u = output_transform(test_data,pre_u)
    test_loss = relative_l2_torch(pre_u,test_u)
    #########test_loss NRMSE#########
    
    #########Record#########
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation.item()) 
    test_loss_1.append(test_loss)
    #########Record#########
    
    if it % 1000 == 0:
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss)
        print(C1)
        
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    it = it + 1   
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss)

It: 0 train_loss: 103.55452728271484 test_loss: 0.6358123421669006
tensor(2., requires_grad=True)
It: 1000 train_loss: 0.3975432515144348 test_loss: 0.3690960705280304
tensor(1.6688, requires_grad=True)
It: 2000 train_loss: 0.16700394451618195 test_loss: 0.29866641759872437
tensor(1.4787, requires_grad=True)
It: 3000 train_loss: 0.03851306438446045 test_loss: 0.14255735278129578
tensor(1.1894, requires_grad=True)
It: 4000 train_loss: 0.0004233501385897398 test_loss: 0.009526378475129604
tensor(1.0106, requires_grad=True)
It: 5000 train_loss: 0.0001041708019329235 test_loss: 0.0026008484419435263
tensor(1.0027, requires_grad=True)
It: 6000 train_loss: 4.981114398106001e-05 test_loss: 0.001403145957738161
tensor(1.0014, requires_grad=True)
It: 7000 train_loss: 2.3611342840013094e-05 test_loss: 0.0010780708398669958
tensor(1.0011, requires_grad=True)
It: 8000 train_loss: 1.175664147012867e-05 test_loss: 0.0006254928885027766
tensor(1.0006, requires_grad=True)
It: 9000 train_loss: 7.884990

In [18]:
import os
folder_path = '../experimental_data/GA_PINN/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

np.save(os.path.join(folder_path, 'loss_T_1.npy'), loss_T_1)
np.save(os.path.join(folder_path, 'test_loss_1.npy'), test_loss_1)
np.save(os.path.join(folder_path, 'C1_list.npy'), C1_list)